In [1]:
%load_ext watermark
%watermark

Last updated: 2024-05-25T14:19:11.593989-05:00

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 8.13.2

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing, feature_extraction


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing, feature_extraction
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
#import category_encoders
from sklearn.model_selection import cross_validate
from category_encoders import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.compose import ColumnTransformer

In [4]:
consumidores = pd.read_csv("C:/Users/Danny/Desktop/Analisis tesis/4 - Pipelines/data/df_consumo_varObjetivo.csv")

In [5]:
consumidores.head()

,departamento,residentes_hogar,d2_04_num_hijos,per_edad_tipo,estrato_tipo,situacion_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo,per_sexo_tipo,...,g_06_a_posibilidad_conseguir_marihuana_tipo,g_06_b_posibilidad_conseguir_cocaina_tipo,g_06_c_posibilidad_conseguir_basuco_tipo,g_07_alguien_ofrecio_comprar_probar_sp_tipo,g_01_a_num_familiares_consumen_sp_imp_tipo,g_02_a_num_amigos_consumen_sp_imp_tipo,g_08_a_ofrecieron_marihuana_imp_tipo,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,CatRiesgo
0,Guaviare,5,0,Joven,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
1,Cordoba,4,3,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
2,Caqueta,4,2,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
3,Bolivar,6,0,Adulto,Estrato alto,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_No_podria_conseguir,g_06_b_No_podria_conseguir,g_06_c_No_podria_conseguir,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
4,Tolima,4,1,Joven,Estrato bajo,Buscando trabajo,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,1


Como vimos en un apartado anterior, en este ejemplo vamos a modificar cada variable en función de su tipo. Al conjunto de pasos que siguen los datos se le llama comúnmente **Pipelines** (literalmente, sistemas de tuberias).

In [6]:
#consumidores.info(verbose=True)

**INCISO**

vamos a modificar dos transformadores de scikitlearn para que sean compatibles con pipelines. Este paso es necesario en la version actual de scikit-learn, pero seguramente será arreglado en el futuro

In [7]:
variable_objetivo = ["CatRiesgo"]
variables_independientes = consumidores.drop(variable_objetivo, axis=1).columns

In [8]:
datos_numericos = consumidores[variables_independientes].select_dtypes([int, float])
col_numericas = datos_numericos.columns
col_no_numericas = consumidores[variables_independientes].select_dtypes([object]).columns

dict_var_ordinales = {
    "per_edad_tipo": ["Adolescente", "Joven", "Adulto", "Persona mayor"],
    "estrato_tipo":["Estrato bajo", "Estrato medio", "Estrato alto"],
    "d_08_estado_salud_tipo":["d_08_Muy_malo","d_08_Malo","d_08_Regular","d_08_Bueno","d_08_Muy_bueno"],
    "frecuencia_consumo_marihuana_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_cocaina_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_basuco_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = consumidores[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = consumidores[col_categoricas]

Un pipeline de sklearn se define como una secuencia de pasos. Cada paso se define con una tupla de forma `[nombre del paso, transformador]`

Por ejemplo, si queremos crear un pipeline que procese las variables numéricas, primero imputándolas y después estandarizandolas, podriamos crear un pipeline

Creamos ahora los pipelines para cada tipo de variable. Es necesario aclarar que se va ha hacer uso de la libreria Category Encoders par ael procesamiento de variables categoricas y categoricas ordinales

In [9]:
transformador_numerico = Pipeline([
    ("imputador_numerico", SimpleImputer(missing_values=np.nan, strategy='mean')),
    ("escalador_numerico", preprocessing.MinMaxScaler())
])

pipeline_numerico = ColumnTransformer([
    ("transformador_numerico", transformador_numerico, col_numericas),
])

pipeline_numerico.fit_transform(consumidores)[:5]

array([[0.30769231, 0.        ],
       [0.23076923, 0.25      ],
       [0.23076923, 0.16666667],
       [0.38461538, 0.        ],
       [0.23076923, 0.08333333]])

In [10]:
pipeline_numerico

ColumnTransformer(transformers=[('transformador_numerico',
                                 Pipeline(steps=[('imputador_numerico',
                                                  SimpleImputer()),
                                                 ('escalador_numerico',
                                                  MinMaxScaler())]),
                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])

Ahora se procede a crear el pipeline categorico

In [11]:
transformador_categorico = ColumnTransformer([
    ("transformador_categorico", OneHotEncoder(handle_unknown="impute"), col_categoricas)
])

pipeline_categorico = Pipeline([
    ("transformador_categorico", transformador_categorico)
])

pipeline_categorico.fit_transform(consumidores)[:5]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
pipeline_categorico

Pipeline(steps=[('transformador_categorico',
                 ColumnTransformer(transformers=[('transformador_categorico',
                                                  OneHotEncoder(handle_unknown='impute'),
                                                  ['departamento',
                                                   'd_12_b_presenta_problema_consumo_sp_barrio_tipo',
                                                   'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo',
                                                   'd_01_aporta_dinero_hogar_tipo',
                                                   'd_11_k_conocimiento_riesgo_cocaina_frecuenteme...
                                                   'g_05_posibilidad_probar_sp_tipo',
                                                   'g_08_c_ofrecieron_basuco_imp_tipo',
                                                   'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                   'd_10_poco_interes_tipo',
                                                   'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                   'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                   'd2_03_estado_civil_tipo',
                                                   'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                   'd2_05_nivel_educativo_tipo',
                                                   'situacion_tipo', ...])]))])

`category_encoders.OrdinalEncoders` requiere un mapeado específico para cada columna, indicando el número por el que queremos reemplazar cada nivel de cada variable ordinal. Si no especificamos un mapa, asume que el orden es aleatorio.

In [13]:
niveles_ordinales = []
for col, levels in dict_var_ordinales.items():
    niveles_ordinales.append({"col": col, 
                              "mapping": dict(zip(levels, range(len(levels))))})

In [14]:
transformador_ordinal = ColumnTransformer([
    ("transformador_ordinal", OrdinalEncoder(mapping=niveles_ordinales), col_ordinales)
])

pipeline_ordinal = Pipeline([
    ("transformador_ordinal", transformador_ordinal)
])
pipeline_ordinal.fit_transform(consumidores)[:5]

array([[1, 0, 1, 0, 0, 0],
       [2, 0, 2, 0, 0, 0],
       [2, 0, 3, 0, 0, 0],
       [2, 2, 3, 0, 0, 0],
       [1, 0, 3, 0, 0, 0]])

Ya tenemos una manera de, dado un conjunto de datos, separarlos y aplicar distintas transformaciones a cada variable. Nos falta una manera de, una vez se han transformado, reunirlas de nuevo.

Para ello podemos usar `FeatureUnion`, que simplemente toma un conjunto de pasos de un pipeline y los une.

In [15]:
pipeline_procesado = FeatureUnion([
    ('variables_numericas', pipeline_numerico),
    ('variables_ordinales', pipeline_ordinal),
    ('variables_categoricas', pipeline_categorico),
])

In [16]:
pipeline_procesado

FeatureUnion(transformer_list=[('variables_numericas',
                                ColumnTransformer(transformers=[('transformador_numerico',
                                                                 Pipeline(steps=[('imputador_numerico',
                                                                                  SimpleImputer()),
                                                                                 ('escalador_numerico',
                                                                                  MinMaxScaler())]),
                                                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                               ('variables_ordinales',
                                Pipeline(steps=[('transformador_ordinal',
                                                 ColumnTransfor...
                                                                                   'g_05_posibilidad_probar_sp_tipo',
                                                                                   'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                   'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                   'd_10_poco_interes_tipo',
                                                                                   'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                   'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                   'd2_03_estado_civil_tipo',
                                                                                   'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                   'd2_05_nivel_educativo_tipo',
                                                                                   'situacion_tipo', ...])]))]))])

In [17]:
consumidores_procesado = pipeline_procesado.fit_transform(consumidores)

Finalmente, necesitamos añadir un estimador al final para predecir en base a los datos transformados, pero primero se separa los datos en datos de entrenamiento y datos de test

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
     consumidores[variables_independientes], consumidores["CatRiesgo"],
     test_size=0.20, random_state=42
)

In [19]:
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report
estimador = svm.SVC(kernel='linear', C=1.0, decision_function_shape='ovr')

In [20]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8000, 39) (8000,)
(2000, 39) (2000,)


Se debe realizar la separacion de datos en datos de entrenamiento y datos de test y ademas se debe establecer el estimador a usar en el modelo

In [21]:
pipeline_estimador = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', estimador)
])

Para el fit se pasan los datos que se obtienen de entrenamiento y para el predict los datos de test

In [22]:
pipeline_estimador.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador', SVC(kernel='linear'))])

In [23]:
predicciones = pipeline_estimador.predict(X_test)

Se evalua el modelo segun lo obtenido

El beneficio de los pipelines, no solo es tener codigo mas legible y poder gestionar de forma ordenada todo el ciclo de vida del modelado, sino que los pipelines tienen todos los beneficios de los objetos de scikitlearn, por ejemplo, podemos usar validacion cruzada directamente con el pipeline.

In [24]:
resultados = {}

def exactitud_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.accuracy_score(y, preds)

def precision_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.precision_score(y, preds, average="macro")

def exhaustividad_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.recall_score(y, preds, average="macro")

def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.f1_score(y, preds, average="macro")

In [25]:
def evaluar_modelo(prmPipeline,X, y, semillas):
    restultados_estimador = {}
    restultados_estimador["exactitud"] = cross_validate(prmPipeline, X, y,
                                           scoring=exactitud_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["precision"] = cross_validate(prmPipeline, X, y,
                                           scoring=precision_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["exhaustividad"] = cross_validate(prmPipeline, X, y,
                                           scoring=exhaustividad_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["f1_score"] = cross_validate(prmPipeline, X, y,
                                           scoring=f1_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    return restultados_estimador

In [26]:

#Funcion para poder visuallizar los resultados de las metricas
def ver_resultados():
    df_res = pd.DataFrame()
    for modelo, metricas in resultados.items():
        for metrica, valores in metricas.items():
            for medida, arreglo in valores.items():
                # Calcula la media de cada array y agrega una nueva fila al DataFrame
                df_res.loc[f"{modelo}_{metrica}", medida] = np.mean(arreglo)
                
    return df_res

In [27]:
estimador_agregado = pipeline_estimador.named_steps['estimador']
resultados["svm"] = evaluar_modelo(estimador_agregado, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
svm_exactitud,4.856020,0.019261,0.984100,0.994629
svm_precision,5.152692,0.025711,0.978580,0.994754
svm_exhaustividad,5.040334,0.024583,0.976901,0.994247
svm_f1_score,5.101464,0.024712,0.977038,0.994497


In [28]:
report = classification_report(y_test, predicciones)
print('Reporte de Clasificación:')
print(report)

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       864
           1       0.97      0.98      0.98       477
           2       0.98      0.98      0.98       475
           3       0.96      0.87      0.91       114
           4       0.92      0.97      0.94        70

    accuracy                           0.98      2000
   macro avg       0.96      0.96      0.96      2000
weighted avg       0.98      0.98      0.98      2000



### Optimizacion de hiperparametros

In [29]:
print(estimador.__doc__)

C-Support Vector Classification.

    The implementation is based on libsvm. The fit time scales at least
    quadratically with the number of samples and may be impractical
    beyond tens of thousands of samples. For large datasets
    consider using :class:`~sklearn.svm.LinearSVC` or
    :class:`~sklearn.linear_model.SGDClassifier` instead, possibly after a
    :class:`~sklearn.kernel_approximation.Nystroem` transformer or
    other :ref:`kernel_approximation`.

    The multiclass support is handled according to a one-vs-one scheme.

    For details on the precise mathematical formulation of the provided
    kernel functions and how `gamma`, `coef0` and `degree` affect each
    other, see the corresponding section in the narrative documentation:
    :ref:`svm_kernels`.

    Read more in the :ref:`User Guide <svm_classification>`.

    Parameters
    ----------
    C : float, default=1.0
        Regularization parameter. The strength of the regularization is
        inversely proport

In [30]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

Grid Básico, este grid es pequeño y rápido de evaluar.

In [31]:
param_distributions = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.01, 0.1],
}

In [33]:
random_search = RandomizedSearchCV(estimator=estimador, param_distributions=param_distributions, n_iter=100, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

In [34]:
pipeline_estimador_op = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search)
])

In [35]:
pipeline_estimador_op.fit(X_train, y_train)

C:\Users\Danny\.conda\envs\data\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 18 is smaller than n_iter=100. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(cv=5, estimator=SVC(kernel='linear'),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': [0.1, 1, 10],
                                                         'gamma': [0.01, 0.1],
                                                         'kernel': ['linear',
                                                                    'rbf',
                                                                    'poly']},
                                    random_state=42, scoring='accuracy'))])

Se crea un pipeline con el mejor estimador obtenido

In [36]:
best_params = random_search.best_params_

In [37]:
pipeline_estimador_op.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [38]:
pipeline_estimador_op_best = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search.best_estimator_)
])

In [39]:
pipeline_estimador_op_best.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador', SVC(C=10, gamma=0.01, kernel='linear'))])

In [40]:
pipeline_estimador_op_best.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [41]:
estimador_best = pipeline_estimador_op_best.named_steps['estimador']
resultados["svm_best"] = evaluar_modelo(estimador_best, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
svm_exactitud,4.856020,0.019261,0.984100,0.994629
svm_precision,5.152692,0.025711,0.978580,0.994754
svm_exhaustividad,5.040334,0.024583,0.976901,0.994247
svm_f1_score,5.101464,0.024712,0.977038,0.994497
svm_best_exactitud,4.393450,0.012650,0.984200,0.998610
svm_best_precision,4.387494,0.015420,0.977627,0.998933
svm_best_exhaustividad,4.358686,0.015691,0.975575,0.998890
svm_best_f1_score,4.367668,0.015700,0.975941,0.998912


### Segunda optimizacion

Grid Moderado, este grid es un poco más grande y toma más combinaciones en cuenta.

In [42]:
param_distributions_2 = {
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.001, 0.01, 0.1, 'scale'],
    'degree': [3, 4], 
}

In [43]:
random_search_2 = RandomizedSearchCV(estimator=estimador, param_distributions=param_distributions_2, n_iter=100, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

In [44]:
pipeline_estimador_op2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_2)
])

In [45]:
pipeline_estimador_op2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(cv=5, estimator=SVC(kernel='linear'),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': [0.01, 0.1, 1, 10,
                                                               100],
                                                         'degree': [3, 4],
                                                         'gamma': [0.001, 0.01,
                                                                   0.1,
                                                                   'scale'],
                                                         'kernel': ['linear',
                                                                    'rbf',
                                                                    'poly']},
                                    random_state=42, scoring='accuracy'))])

In [46]:
best_params2 = random_search_2.best_params_

In [47]:
pipeline_estimador_op2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [48]:
pipeline_estimador_op_best2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_2.best_estimator_)
])

In [49]:
pipeline_estimador_op_best2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador',
                 SVC(C=100, degree=4, gamma=0.1, kernel='linear'))])

In [50]:
pipeline_estimador_op_best2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [51]:
estimador_best2 = pipeline_estimador_op_best2.named_steps['estimador']
resultados["svm_best_2"] = evaluar_modelo(estimador_best2, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
svm_exactitud,4.856020,0.019261,0.984100,0.994629
svm_precision,5.152692,0.025711,0.978580,0.994754
svm_exhaustividad,5.040334,0.024583,0.976901,0.994247
svm_f1_score,5.101464,0.024712,0.977038,0.994497
svm_best_exactitud,4.393450,0.012650,0.984200,0.998610
svm_best_precision,4.387494,0.015420,0.977627,0.998933
svm_best_exhaustividad,4.358686,0.015691,0.975575,0.998890
svm_best_f1_score,4.367668,0.015700,0.975941,0.998912
svm_best_2_exactitud,4.696052,0.011801,0.985500,1.000000
svm_best_2_precision,4.691735,0.014460,0.978681,1.000000


In [52]:
from scipy.stats import uniform

### Tercera optimizacion

Este grid es más exhaustivo y considera más opciones.

In [53]:
param_distributions_3 = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': [0.0001, 0.001, 0.01, 0.1, 'scale'],
    'class_weight': [None, 'balanced'],
    'shrinking': [True, False],
    'decision_function_shape': ['ovr', 'ovo'],
    'degree': [2, 3, 4],  # Para kernel 'poly'
    'coef0': [0.0, 0.1, 0.5, 1]  # Para kernels 'poly' y 'sigmoid'
}

In [54]:
random_search_3 = RandomizedSearchCV(estimator=estimador, param_distributions=param_distributions_3, n_iter=100, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

In [55]:
pipeline_estimador_op3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_3)
])

In [56]:
pipeline_estimador_op3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                 RandomizedSearchCV(cv=5, estimator=SVC(kernel='linear'),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': [0.001, 0.01, 0.1,
                                                               1, 10, 100],
                                                         'class_weight': [None,
                                                                          'balanced'],
                                                         'coef0': [0.0, 0.1,
                                                                   0.5, 1],
                                                         'decision_function_shape': ['ovr',
                                                                                     'ovo'],
                                                         'degree': [2, 3, 4],
                                                         'gamma': [0.0001,
                                                                   0.001, 0.01,
                                                                   0.1,
                                                                   'scale'],
                                                         'kernel': ['linear',
                                                                    'rbf',
                                                                    'poly',
                                                                    'sigmoid'],
                                                         'shrinking': [True,
                                                                       False]},
                                    random_state=42, scoring='accuracy'))])

In [57]:
best_params3 = random_search_3.best_params_

In [58]:
pipeline_estimador_op3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [59]:
pipeline_estimador_op_best3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_3.best_estimator_)
])

In [60]:
pipeline_estimador_op_best3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador',
                 SVC(C=100, class_weight='balanced', coef0=0.5, gamma=0.01,
                     kernel='linear'))])

In [61]:
pipeline_estimador_op_best3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [62]:
estimador_best3 = pipeline_estimador_op_best3.named_steps['estimador']
resultados["svm_best_3"] = evaluar_modelo(estimador_best3, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
svm_exactitud,4.856020,0.019261,0.984100,0.994629
svm_precision,5.152692,0.025711,0.978580,0.994754
svm_exhaustividad,5.040334,0.024583,0.976901,0.994247
svm_f1_score,5.101464,0.024712,0.977038,0.994497
svm_best_exactitud,4.393450,0.012650,0.984200,0.998610
svm_best_precision,4.387494,0.015420,0.977627,0.998933
svm_best_exhaustividad,4.358686,0.015691,0.975575,0.998890
svm_best_f1_score,4.367668,0.015700,0.975941,0.998912
svm_best_2_exactitud,4.696052,0.011801,0.985500,1.000000
svm_best_2_precision,4.691735,0.014460,0.978681,1.000000


### Cuarta optimizacion

Este grid incluye algunos parámetros adicionales para ajustes mas finos.

In [63]:
param_distributions_4 = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.001, 0.01, 0.1, 'scale'],
    'degree': [3, 4],  # Solo relevante para 'poly'
    'coef0': [0.0, 0.1, 0.5],  # Para 'poly' y 'sigmoid'
    'shrinking': [True, False],
    'decision_function_shape': ['ovr'],
    'tol': [1e-3, 1e-2],
}

In [64]:
random_search_4 = RandomizedSearchCV(estimator=estimador, param_distributions=param_distributions_4, n_iter=100, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

In [65]:
pipeline_estimador_op4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_4)
])

In [66]:
pipeline_estimador_op4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(cv=5, estimator=SVC(kernel='linear'),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': [0.1, 1, 10],
                                                         'coef0': [0.0, 0.1,
                                                                   0.5],
                                                         'decision_function_shape': ['ovr'],
                                                         'degree': [3, 4],
                                                         'gamma': [0.001, 0.01,
                                                                   0.1,
                                                                   'scale'],
                                                         'kernel': ['linear',
                                                                    'rbf',
                                                                    'poly'],
                                                         'shrinking': [True,
                                                                       False],
                                                         'tol': [0.001, 0.01]},
                                    random_state=42, scoring='accuracy'))])

In [67]:
best_params4 = random_search_4.best_params_

In [68]:
pipeline_estimador_op4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [69]:
pipeline_estimador_op_best4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_4.best_estimator_)
])

In [70]:
pipeline_estimador_op_best4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador',
                 SVC(C=10, gamma=0.01, kernel='linear', shrinking=False,
                     tol=0.01))])

In [71]:
pipeline_estimador_op_best4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [72]:
estimador_best4 = pipeline_estimador_op_best4.named_steps['estimador']
resultados["svm_best_4"] = evaluar_modelo(estimador_best4, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
svm_exactitud,4.856020,0.019261,0.984100,0.994629
svm_precision,5.152692,0.025711,0.978580,0.994754
svm_exhaustividad,5.040334,0.024583,0.976901,0.994247
svm_f1_score,5.101464,0.024712,0.977038,0.994497
svm_best_exactitud,4.393450,0.012650,0.984200,0.998610
svm_best_precision,4.387494,0.015420,0.977627,0.998933
svm_best_exhaustividad,4.358686,0.015691,0.975575,0.998890
svm_best_f1_score,4.367668,0.015700,0.975941,0.998912
svm_best_2_exactitud,4.696052,0.011801,0.985500,1.000000
svm_best_2_precision,4.691735,0.014460,0.978681,1.000000


### Quinta optimizacion

Grid con Foco en Polinomial y Sigmoide: Este grid es específico para explorar kernels polinomiales y sigmoides.

In [73]:
param_distributions_5 = {
    'C': [0.1, 1, 10],
    'kernel': ['poly', 'sigmoid'],
    'gamma': [0.01, 0.1, 'scale'],
    'degree': [2, 3, 4],
    'coef0': [0.0, 0.5, 1],
    'class_weight': [None, 'balanced'],
}

In [74]:
random_search_5 = RandomizedSearchCV(estimator=estimador, param_distributions=param_distributions_5, n_iter=100, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

In [75]:
pipeline_estimador_op5 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_5)
])

In [76]:
pipeline_estimador_op5.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(cv=5, estimator=SVC(kernel='linear'),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': [0.1, 1, 10],
                                                         'class_weight': [None,
                                                                          'balanced'],
                                                         'coef0': [0.0, 0.5, 1],
                                                         'degree': [2, 3, 4],
                                                         'gamma': [0.01, 0.1,
                                                                   'scale'],
                                                         'kernel': ['poly',
                                                                    'sigmoid']},
                                    random_state=42, scoring='accuracy'))])

In [77]:
best_params5 = random_search_5.best_params_

In [78]:
pipeline_estimador_op5.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [79]:
pipeline_estimador_op_best5 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_5.best_estimator_)
])

In [80]:
pipeline_estimador_op_best5.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador', SVC(C=10, coef0=1, gamma=0.01, kernel='poly'))])

In [81]:
pipeline_estimador_op_best5.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [82]:
estimador_best5 = pipeline_estimador_op_best5.named_steps['estimador']
resultados["svm_best_5"] = evaluar_modelo(estimador_best5, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
svm_exactitud,4.856020,0.019261,0.984100,0.994629
svm_precision,5.152692,0.025711,0.978580,0.994754
svm_exhaustividad,5.040334,0.024583,0.976901,0.994247
svm_f1_score,5.101464,0.024712,0.977038,0.994497
svm_best_exactitud,4.393450,0.012650,0.984200,0.998610
svm_best_precision,4.387494,0.015420,0.977627,0.998933
svm_best_exhaustividad,4.358686,0.015691,0.975575,0.998890
svm_best_f1_score,4.367668,0.015700,0.975941,0.998912
svm_best_2_exactitud,4.696052,0.011801,0.985500,1.000000
svm_best_2_precision,4.691735,0.014460,0.978681,1.000000


### Sexta optimizacion

Grid enfocado en la Regularización y Gamma

In [83]:
param_distributions_6 = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['rbf', 'sigmoid'],
    'gamma': [0.001, 0.01, 0.1, 1],
    'coef0': [0.0, 0.1, 0.5],
    'shrinking': [True, False],
}

In [84]:
random_search_6 = RandomizedSearchCV(estimator=estimador, param_distributions=param_distributions_6, n_iter=100, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

In [85]:
pipeline_estimador_op6 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_6)
])

In [86]:
pipeline_estimador_op6.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(cv=5, estimator=SVC(kernel='linear'),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': [0.01, 0.1, 1,
                                                               10],
                                                         'coef0': [0.0, 0.1,
                                                                   0.5],
                                                         'gamma': [0.001, 0.01,
                                                                   0.1, 1],
                                                         'kernel': ['rbf',
                                                                    'sigmoid'],
                                                         'shrinking': [True,
                                                                       False]},
                                    random_state=42, scoring='accuracy'))])

In [87]:
best_params6 = random_search_6.best_params_

In [88]:
pipeline_estimador_op6.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [89]:
pipeline_estimador_op_best6 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_6.best_estimator_)
])

In [90]:
pipeline_estimador_op_best6.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'd_10_poco_interes_tipo',
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'd2_05_nivel_educativo_tipo',
                                                                                                    'situacion_tipo', ...])]))]))])),
                ('estimador',
                 SVC(C=10, coef0=0.1, gamma=0.01, shrinking=False))])

In [91]:
pipeline_estimador_op_best6.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [92]:
estimador_best6 = pipeline_estimador_op_best6.named_steps['estimador']
resultados["svm_best_6"] = evaluar_modelo(estimador_best6, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
svm_exactitud,4.856020,0.019261,0.984100,0.994629
svm_precision,5.152692,0.025711,0.978580,0.994754
svm_exhaustividad,5.040334,0.024583,0.976901,0.994247
svm_f1_score,5.101464,0.024712,0.977038,0.994497
svm_best_exactitud,4.393450,0.012650,0.984200,0.998610
svm_best_precision,4.387494,0.015420,0.977627,0.998933
svm_best_exhaustividad,4.358686,0.015691,0.975575,0.998890
svm_best_f1_score,4.367668,0.015700,0.975941,0.998912
svm_best_2_exactitud,4.696052,0.011801,0.985500,1.000000
svm_best_2_precision,4.691735,0.014460,0.978681,1.000000


### Exportar con joblib el mejor pipeline obtenido

Tenemos que exportar 3 cosas para poder recrear el Pipeline:

**1. El pipeline entrenado.**

Podemos usar la libreria de scikit-learn `joblib` para guardar nuestro pipeline al disco duro. Joblib es una libreria basada en pickle que sirve para guardar objetos de python al disco duro, pero que está optimizado para scikit-learn.

In [93]:
import joblib

In [95]:
joblib.dump(pipeline_estimador_op_best2, 'pipeline_svm_best.pkl')

['pipeline_svm_best.pkl']

In [96]:
clf = joblib.load('C:/Users/Danny/Desktop/Analisis_Modelo/Analisis Jeffer/Pipelines/Modelo SVM/pipeline_svm_best.pkl')

In [97]:
clf.predict(consumidores.head())

array([0, 0, 0, 0, 1], dtype=int64)